In [5]:
import os
from transformers import AutoTokenizer, MBartForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset
import torch


device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
dataset = load_dataset("json", data_files="train.jsonl", field=None)

tokenizer = AutoTokenizer.from_pretrained('./ru-mbart-large-summ')
model = MBartForConditionalGeneration.from_pretrained('./ru-mbart-large-summ')
tokenizer.src_lang = "ru_RU"
tokenizer.tgt_lang = "ru_RU"

def preprocess_function(examples):
    inputs = examples["text"]
    targets = examples["summary"]
    
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding=True)
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding=True)
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

training_args = TrainingArguments(
    output_dir="./tuned",     
    num_train_epochs=3,                      
    per_device_train_batch_size=2,          
    per_device_eval_batch_size=2,            
    warmup_steps=500,                       
    weight_decay=0.01,                       
    logging_dir="./logs",                    
    logging_steps=50,                        
    evaluation_strategy="steps",             
    eval_steps=500,                          
    save_total_limit=2,                      
    save_steps=500,                          
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    tokenizer=tokenizer,
)

trainer.train()

model.save_pretrained(os.path.join(training_args.output_dir, "final_model"))
tokenizer.save_pretrained(os.path.join(training_args.output_dir, "final_model"))


/opt/homebrew/lib/python3.12/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/72 [00:00<?, ?it/s]

{'loss': 3.022, 'grad_norm': 4.973212718963623, 'learning_rate': 5e-06, 'epoch': 2.08}


/opt/homebrew/lib/python3.12/site-packages/transformers/modeling_utils.py:2618: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 256, 'early_stopping': True, 'num_beams': 10, 'length_penalty': 0.6, 'no_repeat_ngram_size': 2}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


{'train_runtime': 96.0231, 'train_samples_per_second': 1.468, 'train_steps_per_second': 0.75, 'train_loss': 2.9450414975484214, 'epoch': 3.0}


('./tuned/final_model/tokenizer_config.json',
 './tuned/final_model/special_tokens_map.json',
 './tuned/final_model/sentencepiece.bpe.model',
 './tuned/final_model/added_tokens.json',
 './tuned/final_model/tokenizer.json')

In [13]:
%pip install --upgrade accelerate --break-system-packages

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
dataset['train'][:10]

{'text': ['Приложение \n\nк приказу НИУ ВШЭ \nот 30.07.2021 № 6.18.1-01/300721-5\n\nУТВЕРЖДЕН\nученым советом ФЭН\nпротокол № 13 от 29.06.2021\n\n\n\n\nПорядок назначения и выплаты\nстипендий имени Л.Л. Любимова факультета экономических наук \nНационального исследовательского университета «Высшая школа экономики» для студентов и аспирантов, участвующих в долгосрочных программах международной академической мобильности и программах двух дипломов\n\n\nОбщие положения\nПорядок назначения и выплаты стипендий имени Л.Л. Любимова  факультета экономических наук Национального исследовательского университета «Высшая школа экономики» для студентов и аспирантов, участвующих в долгосрочных программах международной академической мобильности и программах двух дипломов (далее соответственно – Порядок, Стипендия, ФЭН, НИУ ВШЭ) определяет критерии отбора получателей, сроки, порядок назначения и выплаты Стипендии для студентов и аспирантов, участвующих в долгосрочных программах международной академическо